# Encoding comparison

This notebook contains a side-by-side evaluation of different bit-to-tree encodings

In [1]:
import sys
sys.path.append('../')
sys.setrecursionlimit(5000) # Croissant

import plotly.graph_objects as go
import numpy as np
import math
import plotly.express as px
import random

from tree_lib.encodings import two_choices
from tree_lib.encodings import counting_ones

#### 0 - Utils

In [2]:

# Generate random bit string, given the probability of picking one
# `p_of_one = 0`   => constant zero
# `p_of_one = 0.5` => pure random
# `p_of_one = 0.9` => mostly ones
# `p_of_one = 1`   => All ones
def gen_bit_string(length, p_of_one=0.5): 
    assert 0 <= p_of_one and p_of_one <= 1
    return ''.join(np.random.choice(["0","1"], length, p=[1-p_of_one, p_of_one]))

# Generate random bit string, given the "auto-correlation with lag 1"
# Autocorrelation represents the degree of similarity between a 
# given time series and a lagged version of itself over successive time intervals.
# `auto_correlation = 0`   => perfectly alternating 
# `auto_correlation = 0.5` => pure random
# `auto_correlation = 1`   => constant string
def gen_bit_string_with_autocorrelation(length, auto_correlation=0.5): # 0.5 = random
    assert 0 <= auto_correlation <= 1 and length > 0
    signal = ["0"]*length
    signal[0] = random.choice("01")
    for i in range(1,length):
        is_same_as_previous = random.random() < auto_correlation
        signal[i] = signal[i-1] if is_same_as_previous else f"{1-int(signal[i-1])}"
    return ''.join(signal)

#### 1 - Define encodings to compare

In [3]:
ENCODINGS = [
    ["two_choices", two_choices.bits_to_tree, two_choices.tree_to_bits],
    ["counting_ones", counting_ones.bits_to_tree, counting_ones.tree_to_bits]
    ]

#### 2. Quick check for correctness

In [4]:

STRINGS_TO_CHECK = [
    "0","1","01","10","11", "100", "101", "110", "111",     # basic
    "010101010101", "0000000000", "1111111111",             # something more complex
    "010101010101"*100, "0000000000"*100, "1111111111"*100, # long strings
]
STRINGS_TO_CHECK += [gen_bit_string(500) for _ in range(500)]

for name, btt, ttb in ENCODINGS:
    for original_string in STRINGS_TO_CHECK:
        decoded = ttb(btt(original_string))
        assert decoded == original_string, f"'{name}' failed: '{original_string}' != '{decoded}'"
    print(f"{name} OK")


two_choices OK
counting_ones OK


#### 3. Generate dataset


In [5]:
import plotly.express as px
import pandas as pd

BIT_STRING_LENGTHS = [10, 50, 100, 200, 500, 700, 1000, 2000]
AUTO_CORRELATIONS = [0., .1, .2, .3, .4, .5, .6, .7, .8, .9, 1.]
SAMPLES_PER_STRING_TYPE = 10

def dataset_for_encoding (encoding_name, bit_to_tree_fun): 
    # Generate random bit strings with various properties
    bit_strings = [gen_bit_string(length) 
                        for length in BIT_STRING_LENGTHS 
                        for _ in range(SAMPLES_PER_STRING_TYPE)]
    bit_strings += [gen_bit_string_with_autocorrelation(length, ac) 
                        for length in BIT_STRING_LENGTHS 
                        for ac in AUTO_CORRELATIONS 
                        for _ in range(SAMPLES_PER_STRING_TYPE)]
    
    # Given a bit string, generate a data point
    # [encoding name, length, num nodes]
    def data_point(bit_str):
        tree = bit_to_tree_fun(bit_str)
        str_len = len(bit_str)
        perc_of_ones = bit_str.count('1') / str_len
        return [encoding_name, str_len, perc_of_ones, tree.n_descendants]
    
    return [data_point(bit_str) for bit_str in bit_strings]

dataset = []
for encoding_name, btt, _ in ENCODINGS:
    dataset += dataset_for_encoding(encoding_name, btt)

df = pd.DataFrame(dataset, columns=['Encoding', 'Bits', 'PercOfOnes', 'Nodes']) 
df


,Encoding,Bits,PercOfOnes,Nodes
0,two_choices,10,0.6,16
1,two_choices,10,0.5,15
2,two_choices,10,0.5,16
3,two_choices,10,0.5,13
4,two_choices,10,0.4,14
...,...,...,...,...
1915,counting_ones,2000,1.0,2002
1916,counting_ones,2000,1.0,2002
1917,counting_ones,2000,1.0,2002
1918,counting_ones,2000,1.0,2002


#### 4. Reports

In [6]:
fig = px.box(df, x="Bits", y="Nodes", color="Encoding", 
             title="Bits vs Number Nodes, Entire Dataset")
fig.show()

df_500 = df[df['Bits'] == 500] # TODO probably need to work more on DB, there is too much data for 0.5 per of ones 
fig = px.box(df_500, x="PercOfOnes", y="Nodes", color="Encoding", 
             title="Percentage of Ones vs Num Nodes, 500 Bits")
fig.show()